## Predicting Exam Performance by Various Factors

**Introduction**

* Relevant background information on the topic

The dataset is obtained in the study “The development of intuitive knowledge classifier and the modeling of domain dependent data” done by H. Tolga Kahraman et. al. (2013) on a web based learning system. The system tracks dynamic data which represent the student’s educational interactions such as visited pages and elapsed time in the concept/goal pages. This dynamic data is used to generate the variables to describe the students and are used to classify users' knowledge.

* The Question we will try to answer with our project

Based on four factors, the degree of study time for material (STG), repetition number for material (SCG), study time for related objects like prerequisites (STR), and exam performance for related objects (LPR), the exam performance for material (PEG), predicting the knowledge level (UNS) and finding a factor that has the strongest relationship with PEG.

* The dataset description that will be used to answer the question

We have two csv files, a training dataset and a testing dataset. The columns in this datasets are STG (the degree of study time for material, a quantitative variable), SCG (The degree of repetition number of user for goal object materials, a quantitative variable), STR (The degree of study time of user for related objects with goal object, a quantitative variable), LPR (The exam performance of user for related objects with goal object, a quantitative variable), PEG (the exam performance for material, a quantitative variable), and UNS (The knowledge level of user, a categorical variable). The training dataset has 258 rows excluding column names and the testing dataset has 145 rows excluding column names.

In [4]:
#Preliminary exploratory data analysis

**Methods**

* Explaination how we will conduct either our data analysis and which variables/columns we will use

First, we find the pairwise correlation of the variables and exclude the variables which are highly correlated. Then, we use the relevant and filtered columns to predict UNS by applying k-nearest neighbors. We will also try to use linear regression on STG, STR, LPR, and LPR to predict PEG and find which variable has the strongest effect on PEG.

* Describe at least one way that you will visualize the results

Using a function ggpairs we would be able to compare the pairwise variables.
We could make a scatterplot.


**Expected outcomes and significance**

* What do you expect to find?

We think that the columns are relevant in influencing the user knowledge and hence we predict that the accuracy of the model that predicts user knowledge level will have a relatively high accuracy level. We also expect that the repetition number for the material will have the highest influence on exam performance.

* What impact could such findings have?

We could suggest a study habit to students based on the results. For example, if there is a high relationship between LPR and PEG, we could recommend studying more about basic stuff in the field.

* What future questions could this lead to?

How to improve the web learning system to accommodate a better growth of knowledge among the users and which metrics (repetition or study time) of which materials (goal materials, or related materials) to focus on improving.





**Reference**

Kahraman, H. T., Sagiroglu, S., & Colak, I. (2013). The development of intuitive knowledge classifier and the modeling of domain dependent data. Knowledge-Based Systems, 37, 283-295. https://doi.org/10.1016/j.knosys.2012.08.009
